<a href="https://colab.research.google.com/github/YuanchengJin/CP1404/blob/master/CodeFormer_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
  <img src="https://user-images.githubusercontent.com/14334509/179359809-bd45566a-486d-418f-83fa-67bbbba8c45c.png" height=120>
</p>

# CodeFormer Inference Demo
## Towards Robust Blind Face Restoration with Codebook Lookup Transformer (NeurIPS 2022)
Shangchen Zhou, Kelvin C.K. Chan, Chongyi Li, Chen Change Loy

[![GitHub Stars](https://img.shields.io/github/stars/sczhou/CodeFormer?style=social)](https://github.com/sczhou/CodeFormer) [![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2206.11253) [![Hugging Face](https://img.shields.io/badge/Demo-%F0%9F%A4%97%20Hugging%20Face-blue)](https://huggingface.co/spaces/sczhou/CodeFormer) ![visitors](https://visitor-badge.glitch.me/badge?page_id=sczhou/CodeFormer)

# 1. Preparations
Before start, make sure that you choose
* Hardware Accelerator = GPU (in the Runtime menu -> Change runtime type)

Then, we clone the repository, set up the envrironment, and download the pre-trained model.

In [9]:
# Clone CodeFormer and enter the CodeFormer folder
%cd /content
!rm -rf CodeFormer
!git clone https://github.com/sczhou/CodeFormer.git
%cd CodeFormer

# Set up the environment
# Install python dependencies
!pip install -r requirements.txt
# Install basicsr
!python basicsr/setup.py develop

# Download the pre-trained model
!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer

# Visualization function
import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('CodeFormer', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

/content
Cloning into 'CodeFormer'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (297/297), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 614 (delta 208), reused 183 (delta 183), pack-reused 317 (from 3)
Receiving objects: 100% (614/614), 17.31 MiB | 34.76 MiB/s, done.
Resolving deltas: 100% (296/296), done.
/content/CodeFormer
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.11/di

&nbsp;
***
&nbsp;
# 2. Test on your images 😀

*   Old photos
*   AI-created face images by DALLE2/Midjourney/Stable Diffusion

If CodeFormer is helpful to your photos, please help star our [repo](https://github.com/sczhou/CodeFormer). Thanks! 🤗

[![GitHub Stars](https://img.shields.io/github/stars/sczhou/CodeFormer?style=social)](https://github.com/sczhou/CodeFormer)

In [14]:
# Upload your own images
import os
from google.colab import files
import shutil

upload_folder = 'inputs/user_upload'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.mkdir(upload_folder)

uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

Saving 000000_00.png to 000000_00.png
move 000000_00.png to inputs/user_upload/000000_00.png


In [17]:
# 简化版本 - 一次性处理
def quick_video_enhancement():
    """快速视频增强"""

    # 1. 上传并拆帧
    video_info = process_video_with_codeformer(max_frames=200, frame_interval=3)

    if video_info:
        print("✅ 帧提取完成，请运行 CodeFormer")
        print("完成后运行: rebuild_video_from_processed_frames()")

    return video_info

# 使用
video_info = quick_video_enhancement()

🎬 视频 -> CodeFormer 处理流程
✅ 创建目录: inputs/user_upload
✅ 创建目录: video_frames
✅ 创建目录: processed_frames
🎬 请上传视频文件...
支持格式: MP4, AVI, MOV, MKV


Saving output (4).mp4 to output (4).mp4
✅ 视频文件: output (4).mp4
🎬 开始提取视频帧...
视频文件: inputs/user_upload/output (4).mp4
📊 视频信息:
   分辨率: 848x624
   帧率: 24.00 FPS
   总帧数: 310
   时长: 12.92 秒
🎯 将提取 103 帧 (间隔: 3)
   已提取 100 帧...
✅ 帧提取完成!
   总共提取: 104 帧
   保存位置: video_frames
📁 准备帧文件用于 CodeFormer 处理...
📋 复制 104 个帧文件...
   已复制 100 个文件...
✅ 帧文件已准备好用于 CodeFormer
   输入目录: inputs/user_upload

✅ 帧提取完成！
🔄 下一步: 运行 CodeFormer 处理这些帧
   然后运行 rebuild_video_from_processed_frames() 重建视频
✅ 帧提取完成，请运行 CodeFormer
完成后运行: rebuild_video_from_processed_frames()


In [18]:
import os
import glob
import subprocess
import sys

def diagnose_codeformer_issues():
    """诊断 CodeFormer 可能的问题"""

    print("=== CODEFORMER 问题诊断 ===\n")

    issues_found = []

    # 1. 检查输入目录和文件
    print("1️⃣ 检查输入目录和文件")
    input_path = "inputs/user_upload"

    if not os.path.exists(input_path):
        print(f"❌ 输入目录不存在: {input_path}")
        issues_found.append("input_directory_missing")

        # 尝试创建目录
        try:
            os.makedirs(input_path, exist_ok=True)
            print(f"✅ 已创建目录: {input_path}")
        except Exception as e:
            print(f"❌ 无法创建目录: {e}")
    else:
        print(f"✅ 输入目录存在: {input_path}")

        # 检查目录中的图像文件
        image_files = glob.glob(os.path.join(input_path, "*"))
        image_files = [f for f in image_files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

        if not image_files:
            print(f"❌ 输入目录中没有图像文件")
            issues_found.append("no_input_images")
        else:
            print(f"✅ 找到 {len(image_files)} 个图像文件:")
            for img in image_files[:3]:  # 显示前3个
                print(f"   - {os.path.basename(img)}")
            if len(image_files) > 3:
                print(f"   - ... 还有 {len(image_files) - 3} 个文件")

    print()

    # 2. 检查 CodeFormer 脚本
    print("2️⃣ 检查 CodeFormer 脚本")
    script_path = "inference_codeformer.py"

    if not os.path.exists(script_path):
        print(f"❌ CodeFormer 脚本不存在: {script_path}")
        issues_found.append("script_missing")

        # 检查其他可能的位置
        possible_paths = [
            "./CodeFormer/inference_codeformer.py",
            "./codeformer/inference_codeformer.py",
            "./scripts/inference_codeformer.py"
        ]

        for path in possible_paths:
            if os.path.exists(path):
                print(f"✅ 找到脚本: {path}")
                print(f"   建议使用: python {path}")
                break
    else:
        print(f"✅ CodeFormer 脚本存在: {script_path}")

    print()

    # 3. 检查模型文件
    print("3️⃣ 检查预训练模型")
    model_dirs = [
        "./weights",
        "./models",
        "./CodeFormer/weights",
        "./checkpoints"
    ]

    model_found = False
    for model_dir in model_dirs:
        if os.path.exists(model_dir):
            model_files = glob.glob(os.path.join(model_dir, "*.pth"))
            if model_files:
                print(f"✅ 找到模型文件在: {model_dir}")
                for model in model_files:
                    size_mb = os.path.getsize(model) / (1024*1024)
                    print(f"   - {os.path.basename(model)} ({size_mb:.1f} MB)")
                model_found = True
                break

    if not model_found:
        print("❌ 未找到预训练模型文件")
        issues_found.append("models_missing")

    print()

    # 4. 检查 Python 依赖
    print("4️⃣ 检查 Python 依赖")
    required_packages = [
        'torch', 'torchvision', 'opencv-python', 'pillow',
        'numpy', 'basicsr', 'facexlib', 'gfpgan'
    ]

    missing_packages = []
    for package in required_packages:
        try:
            __import__(package.replace('-', '_'))
            print(f"✅ {package}")
        except ImportError:
            print(f"❌ {package} - 未安装")
            missing_packages.append(package)

    if missing_packages:
        issues_found.append("dependencies_missing")

    print()

    # 5. 检查 GPU 支持
    print("5️⃣ 检查 GPU 支持")
    try:
        import torch
        if torch.cuda.is_available():
            print(f"✅ CUDA 可用: {torch.cuda.get_device_name(0)}")
            print(f"   GPU 内存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
        else:
            print("⚠️  CUDA 不可用，将使用 CPU（速度较慢）")
    except ImportError:
        print("❌ PyTorch 未安装")

    return issues_found, missing_packages

def provide_solutions(issues_found, missing_packages):
    """提供解决方案"""

    print("\n=== 🛠️  解决方案 ===\n")

    if "input_directory_missing" in issues_found or "no_input_images" in issues_found:
        print("📁 输入文件问题解决方案:")
        print("```python")
        print("# 创建输入目录")
        print("import os")
        print("os.makedirs('inputs/user_upload', exist_ok=True)")
        print("")
        print("# 将你的图像文件复制到这个目录")
        print("# 或者修改命令中的 --input_path 参数")
        print("```")
        print()

    if "script_missing" in issues_found:
        print("📜 脚本文件问题解决方案:")
        print("```bash")
        print("# 下载 CodeFormer")
        print("git clone https://github.com/sczhou/CodeFormer.git")
        print("cd CodeFormer")
        print("```")
        print()

    if "models_missing" in issues_found:
        print("🤖 模型文件问题解决方案:")
        print("```bash")
        print("# 下载预训练模型")
        print("python scripts/download_pretrained_models.py facelib")
        print("python scripts/download_pretrained_models.py CodeFormer")
        print("# 或者手动下载模型文件到 weights/ 目录")
        print("```")
        print()

    if "dependencies_missing" in issues_found:
        print("📦 依赖包问题解决方案:")
        print("```bash")
        print("# 安装缺失的包")
        for package in missing_packages:
            print(f"pip install {package}")
        print("")
        print("# 或者使用 requirements.txt")
        print("pip install -r requirements.txt")
        print("```")
        print()

def create_fixed_inference_code():
    """创建修复后的推理代码"""

    print("=== 🔧 修复后的推理代码 ===\n")

    code = '''
# 修复后的 CodeFormer 推理代码
import os
import subprocess
import glob

# 配置参数
CODEFORMER_FIDELITY = 0.7
BACKGROUND_ENHANCE = True
FACE_UPSAMPLE = False

# 1. 检查和创建输入目录
input_dir = "inputs/user_upload"
os.makedirs(input_dir, exist_ok=True)

# 2. 检查输入文件
input_files = glob.glob(os.path.join(input_dir, "*"))
image_files = [f for f in input_files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

if not image_files:
    print("❌ 错误: 输入目录中没有图像文件")
    print(f"请将图像文件放入: {input_dir}")
    exit(1)

print(f"✅ 找到 {len(image_files)} 个输入图像")

# 3. 构建命令
base_cmd = f"python inference_codeformer.py -w {CODEFORMER_FIDELITY} --input_path {input_dir}"

if BACKGROUND_ENHANCE:
    base_cmd += " --bg_upsampler realesrgan"

if FACE_UPSAMPLE:
    base_cmd += " --face_upsample"

# 4. 执行命令
print(f"执行命令: {base_cmd}")

try:
    result = subprocess.run(base_cmd, shell=True, capture_output=True, text=True)

    if result.returncode == 0:
        print("✅ CodeFormer 执行成功!")
        print("输出文件位置:")

        # 查找输出文件
        output_dirs = ["./results", "./outputs"]
        for output_dir in output_dirs:
            if os.path.exists(output_dir):
                output_files = glob.glob(os.path.join(output_dir, "**/*.jpg"), recursive=True)
                output_files.extend(glob.glob(os.path.join(output_dir, "**/*.png"), recursive=True))

                for output_file in output_files:
                    print(f"  📸 {output_file}")
    else:
        print("❌ CodeFormer 执行失败:")
        print("错误输出:", result.stderr)

except Exception as e:
    print(f"❌ 执行错误: {e}")
'''

    return code

# 运行诊断
if __name__ == "__main__":
    issues, missing_deps = diagnose_codeformer_issues()
    provide_solutions(issues, missing_deps)

    print("\n" + "="*50)
    fixed_code = create_fixed_inference_code()
    print(fixed_code)

=== CODEFORMER 问题诊断 ===

1️⃣ 检查输入目录和文件
✅ 输入目录存在: inputs/user_upload
✅ 找到 104 个图像文件:
   - frame_000077.jpg
   - frame_000063.jpg
   - frame_000072.jpg
   - ... 还有 101 个文件

2️⃣ 检查 CodeFormer 脚本
✅ CodeFormer 脚本存在: inference_codeformer.py

3️⃣ 检查预训练模型
❌ 未找到预训练模型文件

4️⃣ 检查 Python 依赖
✅ torch
✅ torchvision
❌ opencv-python - 未安装
❌ pillow - 未安装
✅ numpy
✅ basicsr
❌ facexlib - 未安装
❌ gfpgan - 未安装

5️⃣ 检查 GPU 支持
⚠️  CUDA 不可用，将使用 CPU（速度较慢）

=== 🛠️  解决方案 ===

🤖 模型文件问题解决方案:
```bash
# 下载预训练模型
python scripts/download_pretrained_models.py facelib
python scripts/download_pretrained_models.py CodeFormer
# 或者手动下载模型文件到 weights/ 目录
```

📦 依赖包问题解决方案:
```bash
# 安装缺失的包
pip install opencv-python
pip install pillow
pip install facexlib
pip install gfpgan

# 或者使用 requirements.txt
pip install -r requirements.txt
```


=== 🔧 修复后的推理代码 ===


# 修复后的 CodeFormer 推理代码
import os
import subprocess
import glob

# 配置参数
CODEFORMER_FIDELITY = 0.7
BACKGROUND_ENHANCE = True
FACE_UPSAMPLE = False

# 1. 检查和创建输入目录
input_dir = "inputs

In [15]:
# Inference the uploaded images
#@markdown `CODEFORMER_FIDELITY`: Balance the quality (lower number) and fidelity (higher number)<br>
# you can add '--bg_upsampler realesrgan' to enhance the background
CODEFORMER_FIDELITY = 0.7 #@param {type:"slider", min:0, max:1, step:0.01}
#@markdown `BACKGROUND_ENHANCE`: Enhance background image with Real-ESRGAN<br>
BACKGROUND_ENHANCE = True #@param {type:"boolean"}
#@markdown `FACE_UPSAMPLE`: Upsample restored faces for high-resolution AI-created images<br>
FACE_UPSAMPLE = False #@param {type:"boolean"}
if BACKGROUND_ENHANCE:
  if FACE_UPSAMPLE:
    !python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path inputs/user_upload --bg_upsampler realesrgan --face_upsample
  else:
    !python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path inputs/user_upload --bg_upsampler realesrgan
else:
  !python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path inputs/user_upload

Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/RealESRGAN_x2plus.pth" to /content/CodeFormer/weights/realesrgan/RealESRGAN_x2plus.pth

100% 64.0M/64.0M [00:00<00:00, 156MB/s]
/content/CodeFormer/inference_codeformer.py:49: RuntimeWarning: Running on CPU now! Make sure your PyTorch version matches your CUDA.The unoptimized RealESRGAN is slow on CPU. If you want to disable it, please remove `--bg_upsampler` and `--face_upsample` in command.
  warnings.warn('Running on CPU now! Make sure your PyTorch version matches your CUDA.'
Face detection model: retinaface_resnet50
Background upsampling: True, Face upsampling: False
[1/1] Processing: 000000_00.png
	detect 1 faces

All results are saved in results/user_upload_0.7


In [12]:
# Visualize the results
import os
import glob

input_folder = 'inputs/user_upload'
result_folder = f'results/user_upload_{CODEFORMER_FIDELITY}/final_results'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
for input_path in input_list:
  img_input = imread(input_path)
  basename = os.path.splitext(os.path.basename(input_path))[0]
  output_path = os.path.join(result_folder, basename+'.png')
  img_output = imread(output_path)
  display(img_input, img_output)

error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
# Download results
!ls results
print('Download results')
os.system(f'zip -r results.zip results/user_upload_{CODEFORMER_FIDELITY}/final_results')
files.download("results.zip")

&nbsp;
***
&nbsp;
## 3. Inference on the demo images (whole images)

In [ ]:
# We set w to 0.7 for the whole images
# you can add '--bg_upsampler realesrgan' to enhance the background
CODEFORMER_FIDELITY = 0.7
!python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path inputs/whole_imgs --bg_upsampler realesrgan

In [ ]:
# Visualize the results
import os
import glob

input_folder = 'inputs/whole_imgs'
result_folder = f'results/whole_imgs_{w}/final_results'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)

&nbsp;
***
&nbsp;
# 4. Inference on the demo faces (croped and aligned faces)

In [ ]:
!rm -rf results

# We set w to 0.5 for the cropped and aligned faces
CODEFORMER_FIDELITY = 0.5
!python inference_codeformer.py -w $CODEFORMER_FIDELITY --has_aligned --input_path inputs/cropped_faces

In [ ]:
# Visualize the results
import os
import glob

input_folder = 'inputs/cropped_faces'
result_folder = f'results/cropped_faces_{w}/restored_faces'
input_list = sorted(glob.glob(os.path.join(input_folder, '*')))
output_list = sorted(glob.glob(os.path.join(result_folder, '*')))
for input_path, output_path in zip(input_list, output_list):
  img_input = imread(input_path)
  img_output = imread(output_path)
  display(img_input, img_output)